In [45]:
from datasets import load_dataset
from flair.data import Sentence
from flair.models import SequenceTagger
from tqdm import tqdm
import numpy as np


In [2]:
dataset = load_dataset("conll2003", split="test", version="1.0.0")
tagger = SequenceTagger.load("flair/ner-english")

_CONLL_TAGS = {
    'PER': 1,
    'ORG': 3,
    'LOC': 5,
    'MISC': 7,
}

Using custom data configuration default
Reusing dataset conll2003 (/home/yxchen/.cache/huggingface/datasets/conll2003/default/1.0.0/40e7cb6bcc374f7c349c83acd1e9352a4f09474eb691f64f364ee62eb65d0ca6)


2021-09-19 19:20:09,382 loading file /home/yxchen/.flair/models/ner-english/4f4cdab26f24cb98b732b389e6cebc646c36f54cfd6e0b7d3b90b25656e4262f.8baa8ae8795f4df80b28e7f7b61d788ecbb057d1dc85aacb316f1bd02837a4a4


In [36]:
sentence = Sentence("George Waschington was the president of the USA")
tagger.predict(sentence)

8

In [57]:
labels, predictions, valid = [], [], 0
for record in tqdm(dataset):
    tokens, ner_tags = record['tokens'], record['ner_tags']
    sentence = Sentence(" ".join(tokens))
    
    if len(sentence) != len(tokens):
        continue
    valid += 1
    labels += ner_tags

    pred = np.array([0] * len(ner_tags))
    tagger.predict(sentence)

    for span in sentence.get_spans('ner'):
        span_pos = span.position_string.split('-')
        span_tag = span.tag

        span_start = int(span_pos[0]) - 1
        pred[span_start] = _CONLL_TAGS[span.tag]

        if len(span) == 2:
            span_end = int(span_pos[1]) - 1
            pred[(span_start+1): (span_end+1)] = _CONLL_TAGS[span.tag] + 1
    pred = list(pred)
    assert len(pred) == len(ner_tags)
    predictions += pred
print(valid)

100%|██████████| 3453/3453 [03:20<00:00, 17.24it/s]

2937


In [58]:
import sklearn
sklearn.metrics.f1_score(labels, predictions, labels=list(range(1, 9)), average='micro')

0.8924994084706996